In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
input_path = 'output_data/cities.csv'
weather_df = pd.read_csv(input_path)
weather_df = weather_df.drop(columns="Unnamed: 0")
weather_df.head(10)

,Cities,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,rikitea,-23.1203,-134.9692,75.16,79.0,90.0,18.61,1.635035e+09
1,half moon bay,37.4636,-122.4286,68.18,71.0,90.0,4.00,1.635035e+09
2,aklavik,68.2191,-135.0107,30.92,94.0,46.0,3.11,1.635035e+09
3,kodiak,57.7900,-152.4072,45.91,76.0,90.0,10.36,1.635035e+09
4,ponta do sol,32.6667,-17.1000,69.01,65.0,21.0,6.89,1.635035e+09
5,mataura,-46.1927,168.8643,58.46,80.0,100.0,1.30,1.635035e+09
6,provideniya,64.3833,-173.3000,25.81,77.0,100.0,8.21,1.635035e+09
7,bluff,-46.6000,168.3333,52.30,92.0,100.0,1.81,1.635035e+09
8,cururupu,-1.8283,-44.8683,76.33,89.0,100.0,3.18,1.635035e+09
9,vaini,-21.2000,-175.2000,78.96,73.0,20.0,11.50,1.635035e+09


## Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
location = weather_df[['Lat', 'Lng']]
humidity = weather_df.Humidity

gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, point_radius=10)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))